In [1]:
!pip install optuna==1.4.0
!pip install PyMySQL==0.9.3

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [1]:
import boto3 # AWS Python SDK
import numpy as np
import optuna

# obtain parameters from CloudFormation Outputs
stack_name = 'optuna-lstm'

client = boto3.client('cloudformation')
outputs = client.describe_stacks(StackName=stack_name)['Stacks'][0]['Outputs']

host = [out['OutputValue'] for out in outputs if out['OutputKey'] == 'ClusterEndpoint'][0].split(':')[0]
db_name = [out['OutputValue'] for out in outputs if out['OutputKey'] == 'DatabaseName'][0]
secret_name = [out['OutputValue'] for out in outputs if out['OutputKey'] == 'DBSecretArn'][0].split(':')[-1].split('-')[0]

subnets = [out['OutputValue'] for out in outputs if out['OutputKey'] == 'PrivateSubnets'][0].split(',')
security_group_ids = [out['OutputValue'] for out in outputs if out['OutputKey'] == 'SageMakerSecurityGroup'][0].split(',')

In [2]:
# Call AWS Secrets Manager
from sage.secrets import get_secret
region_name = boto3.session.Session().region_name
secret = get_secret(secret_name, region_name)

# PyMySQL https://docs.sqlalchemy.org/en/13/dialects/mysql.html#module-sqlalchemy.dialects.mysql.pymysql   
db = 'mysql+pymysql://{}:{}@{}/{}'.format(secret['username'], secret['password'], host, db_name)

In [3]:
# Setup
from sagemaker import get_execution_role
import sagemaker

sagemaker_session = sagemaker.Session()

# This role retrieves the SageMaker-compatible role used by this notebook instance.
role = get_execution_role()

In [4]:
# create study in RDS/Aurora
study_name = 'pytorch-lstm'
optuna.study.create_study(storage=db, study_name=study_name, direction='minimize', load_if_exists=True)

[I 2020-09-23 21:48:06,867] Using an existing study with name 'pytorch-lstm' instead of creating a new one.


In [5]:
input_data = sagemaker_session.upload_data(path='../data')

In [6]:
# setup SageMaker PyTorch estimator
from sagemaker.pytorch.estimator import PyTorch

pytorch_estimator = PyTorch(entry_point='pytorch_lstm.py',
                            source_dir='sage',
                            framework_version='1.5.0', 
                            role=role,
                            subnets=subnets,
                            security_group_ids=security_group_ids,
                            train_instance_count=1,
                            train_instance_type='ml.m5.xlarge',
                            hyperparameters={
                                'host': host, 
                                'db-name': db_name, 
                                'db-secret': secret_name, 
                                'study-name': study_name, 
                                'region-name': region_name, 
                                'n-trials': 25
                            })

In [13]:
# HPO in parallel
max_parallel_jobs = 4

for j in range(max_parallel_jobs-1):
    pytorch_estimator.fit(input_data, wait=False)
pytorch_estimator.fit(input_data)

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'Tr

2020-09-23 22:36:57 Starting - Starting the training job...
2020-09-23 22:36:58 Starting - Launching requested ML instances......
2020-09-23 22:38:07 Starting - Preparing the instances for training...
2020-09-23 22:38:40 Downloading - Downloading input data...
2020-09-23 22:39:24 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-09-23 22:39:25,081 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-09-23 22:39:25,084 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-09-23 22:39:25,095 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-09-23 22:39:25,098 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-09-23 22:39:25,360 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 

UnexpectedStatusException: Error for Training job pytorch-training-2020-09-23-22-36-56-028: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/opt/conda/bin/python pytorch_lstm.py --db-name optuna --db-secret demo/optuna/db --host optuna-parameter.cluster-cwu2tv1go0fi.us-east-2.rds.amazonaws.com --n-trials 25 --region-name us-east-2 --study-name pytorch-lstm"
[W 2020-09-23 22:41:51,116] Setting status of trial#13 as TrialState.FAIL because of the following error: RuntimeError('input.size(-1) must be equal to input_size. Expected 74, got 73',)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/optuna/study.py", line 699, in _run_trial
    result = func(trial)
  File "pytorch_lstm.py", line 134, in objective
    predictions = model(seasons)
  File "/opt/conda/lib/python3.6/site-packages/torch/nn/modules/module.py", line 561, in __call__
    result = self.forward(*input, **kwargs)
  File "pytorch_lstm.py", line 95, in forward
    out, hidden =